In [1]:
import sys
sys.path.insert(1, '../SyMBac/')

from SyMBac.drawing import raster_cell
from SyMBac.PSF import PSF_generator
from SyMBac.renderer import convolve_rescale
import numpy as np
from tqdm.auto import tqdm
from PIL import Image
from skimage.util import img_as_uint
import os
from joblib import Parallel, delayed
from skimage.transform import rotate
from SyMBac.PSF import Camera

In [2]:
pad_amount = 200

radius = 150
wavelength = 0.6
NA = 1.45
n = 1.518
resize_amount = 1
pix_scaler = 1
pix_mic_conv = 0.065 / pix_scaler
apo_sigma = 11

FL_PSF = PSF_generator(
    radius = radius,
    wavelength = wavelength,
    NA = NA,
    n = n,
    resize_amount = resize_amount,
    pix_mic_conv = pix_mic_conv,
    apo_sigma = apo_sigma,
    mode="3d fluo",
    condenser = "Ph3",
    z_height = 22, 
    pz = 0.5,
    working_distance=170
)
FL_PSF.calculate_PSF()
FL_PSF.kernel = np.sum(FL_PSF.kernel, axis=0)

In [3]:
def generate_agar_pad_image(border, img_size, density, iter_threshold, min_length, max_length, min_width, max_width):
    image = np.zeros((int(img_size*pix_scaler * border), int(img_size*pix_scaler * border)))
    mask = np.zeros((int(img_size*pix_scaler * border), int(img_size*pix_scaler * border)))
    mask_labaled = np.zeros((int(img_size*pix_scaler * border), int(img_size*pix_scaler * border))).astype(np.uint)
     #DENSITY In cells per square micron
    n_cells = int(density * ((pix_mic_conv * img_size) ** 2))
    mask_counter = 0
    iterations = 0
    stop_condition = int(n_cells*iter_threshold)
    #pbar = tqdm(total=stop_condition)
    cell_info = []
    while (mask_counter != n_cells) and (iterations != stop_condition):

        cell_length = np.random.uniform(min_length, max_length)
        cell_width = np.random.uniform(min_width, max_width)

        raster_cell_length = cell_length/pix_mic_conv * resize_amount
        raster_cell_width = cell_width/pix_mic_conv * resize_amount

        cell = raster_cell(length=raster_cell_length, width=raster_cell_width, separation=0)
        cell = rotate(cell, angle=np.random.uniform(0,365), resize=True, order = 0, preserve_range = True)
        cell *= int(np.random.uniform(1300,1400))
        #

        cell = cell.astype(int)
        cell_mask = cell > 0
        pos_x_start = int(np.random.uniform(0, image.shape[1] - cell.shape[1]))
        pos_y_start = int(np.random.uniform(0, image.shape[0] - cell.shape[0]))
        if mask[pos_y_start + cell.shape[0]//2, pos_x_start + cell.shape[1]//2]:
            pass
        else:

            mask[pos_y_start:pos_y_start+cell.shape[0], pos_x_start:pos_x_start+cell.shape[1]] += cell_mask
            if np.sum(mask > 1) > 0:
                mask[pos_y_start:pos_y_start+cell.shape[0], pos_x_start:pos_x_start+cell.shape[1]] -= cell_mask
            else:
                image[pos_y_start:pos_y_start+cell.shape[0], pos_x_start:pos_x_start+cell.shape[1]] += cell
                mask_labaled[pos_y_start:pos_y_start+cell.shape[0], pos_x_start:pos_x_start+cell.shape[1]] += (cell_mask * mask_counter).astype(np.uint)
                cell_info.append([mask_counter, cell_length, cell_width])
                mask_counter += 1
                
        iterations += 1
        #pbar.update(1)
    #pbar.close()

    return image, mask_labaled, cell_info

In [4]:
densities = [0.15]*200 #fix the density of FPs in cells this time

In [5]:
try:
    os.mkdir("data")
except:
    pass
min_max_cell_length = [ (1.5,3), (2,4), (3,6), (4,8), (5,10), (6,12) ]
min_max_cell_width = [ (0.7,0.7), (0.8,0.8), (0.9,0.9), (1,1), (1.1,1.1), (1.2,1.2) ]


In [6]:
list(zip(min_max_cell_length, min_max_cell_width))

[((1.5, 3), (0.7, 0.7)),
 ((2, 4), (0.8, 0.8)),
 ((3, 6), (0.9, 0.9)),
 ((4, 8), (1, 1)),
 ((5, 10), (1.1, 1.1)),
 ((6, 12), (1.2, 1.2))]

In [7]:
for (min_length, max_length), (min_width, max_width) in zip(min_max_cell_length, min_max_cell_width):
    try:
        os.mkdir(f"data/")
    except:
        pass
    try:
        os.mkdir(f"data/{min_length}_{max_length}")
    except:
        pass
    try:
        os.mkdir(f"data/{min_length}_{max_length}/training_data")
    except:
        pass
    try:
        os.mkdir(f"data/{min_length}_{max_length}/test_data")
    except:
        pass
    try:
        os.mkdir(f"data/{min_length}_{max_length}/raw_train")
    except:
        pass
    try:
        os.mkdir(f"data/{min_length}_{max_length}/raw_test")
    except:
        pass
    img_mask_pairs = Parallel(n_jobs=-1)(delayed(generate_agar_pad_image)(1, 800, density, 2, min_length, max_length, min_width, max_width) for density in tqdm(densities))

    
    conv_imgs = []
    for a, (image, mask, cell_info) in enumerate(tqdm(img_mask_pairs)):
        my_camera = Camera(baseline=100, sensitivity=2.9, dark_noise=8)
        dark_image = my_camera.render_dark_image(size=image.shape, plot=False);
        conv_image = convolve_rescale(image, FL_PSF.kernel, rescale_factor=1/pix_scaler, rescale_int=True)
        conv_image = (dark_image + conv_image * 1000)
        conv_image /= conv_image.max()
        if a < 20:
            Image.fromarray(img_as_uint(conv_image/conv_image.max())).save(f"data/{min_length}_{max_length}/training_data/{str(a).zfill(4)}.png")
            Image.fromarray(img_as_uint(mask)).save(f"data/{min_length}_{max_length}/training_data/{str(a).zfill(4)}_masks.png")
            Image.fromarray(img_as_uint(image/image.max())).save(f"data/{min_length}_{max_length}/raw_train/{str(a).zfill(4)}.png")
        else:
            Image.fromarray(img_as_uint(conv_image/conv_image.max())).save(f"data/{min_length}_{max_length}/test_data/{str(a).zfill(4)}.png")
            Image.fromarray(img_as_uint(mask)).save(f"data/{min_length}_{max_length}/test_data/{str(a).zfill(4)}_masks.png")
            Image.fromarray(img_as_uint(image/image.max())).save(f"data/{min_length}_{max_length}/raw_test/{str(a).zfill(4)}.png")

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

/home/gh464/miniconda3/envs/symbac/lib/python3.10/site-packages/skimage/util/dtype.py:501: UserWarning: Downcasting uint64 to uint16 without scaling because max value 404 fits in uint16
  return _convert(image, np.uint16, force_copy)


  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

...

  0%|          | 0/200 [00:00<?, ?it/s]

/home/gh464/miniconda3/envs/symbac/lib/python3.10/site-packages/skimage/util/dtype.py:501: UserWarning: Downcasting uint64 to uint16 without scaling because max value 100 fits in uint16
  return _convert(image, np.uint16, force_copy)
/home/gh464/miniconda3/envs/symbac/lib/python3.10/site-packages/skimage/util/dtype.py:501: UserWarning: Downcasting uint64 to uint16 without scaling because max value 106 fits in uint16
  return _convert(image, np.uint16, force_copy)
/home/gh464/miniconda3/envs/symbac/lib/python3.10/site-packages/skimage/util/dtype.py:501: UserWarning: Downcasting uint64 to uint16 without scaling because max value 102 fits in uint16
  return _convert(image, np.uint16, force_copy)
/home/gh464/miniconda3/envs/symbac/lib/python3.10/site-packages/skimage/util/dtype.py:501: UserWarning: Downcasting uint64 to uint16 without scaling because max value 104 fits in uint16
  return _convert(image, np.uint16, force_copy)
/home/gh464/miniconda3/envs/symbac/lib/python3.10/site-packages/

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

/home/gh464/miniconda3/envs/symbac/lib/python3.10/site-packages/skimage/util/dtype.py:501: UserWarning: Downcasting uint64 to uint16 without scaling because max value 78 fits in uint16
  return _convert(image, np.uint16, force_copy)
/home/gh464/miniconda3/envs/symbac/lib/python3.10/site-packages/skimage/util/dtype.py:501: UserWarning: Downcasting uint64 to uint16 without scaling because max value 84 fits in uint16
  return _convert(image, np.uint16, force_copy)
/home/gh464/miniconda3/envs/symbac/lib/python3.10/site-packages/skimage/util/dtype.py:501: UserWarning: Downcasting uint64 to uint16 without scaling because max value 91 fits in uint16
  return _convert(image, np.uint16, force_copy)
/home/gh464/miniconda3/envs/symbac/lib/python3.10/site-packages/skimage/util/dtype.py:501: UserWarning: Downcasting uint64 to uint16 without scaling because max value 80 fits in uint16
  return _convert(image, np.uint16, force_copy)
/home/gh464/miniconda3/envs/symbac/lib/python3.10/site-packages/skim